In [34]:
import pandas as pd
import numpy as np
import ta 
from sklearn.linear_model import LogisticRegression
import math
pd.options.mode.chained_assignment = None


#dados foram baixados de https://www.cryptodatadownload.com/ 
#indicadores tecnicos de https://www.akademiabaru.com/doc/ARBMSV14_N1_P35_41.pdf
#indicadores são:  MACD, RSI, SO, MA, EMA, ROC e VT.


df = pd.read_csv("Binance_BTCUSDT_minute.csv",skiprows=1,nrows=100) #lendo somente as 10000 primeiras linhas para salvar processamento enquanto o codigo é desenvolvido
#lembrar de remover ultimas 74 rows 

row_70 = math.floor(0.7*len(df))


#dataframe com 70% das linhas do banco de dados para treinar o modelo, determinar os coeficientes de regressão
df_train = df.iloc[:row_70]

#dataframe com as linhas restantes para testar o modelo
df_test = df.iloc[row_70:]


#calculando os indicadores

#1 - MACD 
df_train["MACD"] = ta.trend.macd(close=df_train.close) # Nas primeiras 26 linhas MACD retorna nan, pois precisa de 26 elementos.

#2 - RSI lembrar de ajustar window para melhorar resultados. Lembrar que nas primeiras 14 linhas RSI = nan, pois ele precisa de 14 elementos anteriores.
RSI_window = 14
df_train["RSI"] = ta.momentum.rsi(df_train.close,window=RSI_window)


#3 - stochastic oscillator (SO)
SO_window = 14
df_train["SO"] = ta.momentum.stoch(close=df_train.close,high=df_train.high,low=df_train.low,window=SO_window)

#3 - Simple moving average (SMA)

df_train["12_SMA"] = ta.trend.sma_indicator(close=df_train.close,window=12)
df_train["26_SMA"] = ta.trend.sma_indicator(close=df_train.close,window=26)

#4 - Exponential moving average (EMA)

df_train["12_EMA"] = ta.trend.ema_indicator(close=df_train.close,window=12)
df_train["26_EMA"] = ta.trend.ema_indicator(close=df_train.close,window=26)

#5 - Rate of change (ROC)
ROC_window = 24 #definir window direito (minima ideia qual window tem que meter aqui)
df_train["ROC"] = ta.momentum.roc(close=df_train.close,window=ROC_window)


#6 - indicadores de volume, mineração de btc e remuneração dos mineradores

# adicionando coluna que possui 0, caso o ultimo close seja > que o close atual e 1 caso o contrário

change = [0]
x = [] #para de ter nan a partir de 24
for i in range(0,len(df_train)):
    x.append([df_train.iloc[i].MACD,df_train.iloc[i].SO,df_train.iloc[i].RSI,df_train.iloc[i]["12_SMA"],df_train.iloc[i]["12_EMA"],df_train.iloc[i].ROC])
    if i != 0:
        diff = df_train.iloc[i].close- df_train.iloc[i-1].close
        if diff >= 0 :
            change.append(1)
        if diff < 0:
            change.append(0)

df_train["change"] = change


#é necessário ignorar as primeiras 25 linhas do dataframe pois alguns indicadores não possuem valores suficientes antes de si e, portanto, assumem valores Nan.
y = list(df_train["change"])[25:]
x = x[25:]


model = LogisticRegression()
model.fit(x,y)
coeficientes = list(model.coef_[0])
coeficientes



70
30


C:\Users\marce\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[-0.17788770955276761,
 -0.011084236237128885,
 0.7804994278204458,
 0.1533690864754773,
 -0.1539574489132077,
 0.036331484860568745]